## École Polytechnique de Montréal
## Département Génie Informatique et Génie Logiciel

## INF8460 – Traitement automatique de la langue naturelle - TP2

## Objectifs d'apprentissage: 

•	Explorer les modèles d’espace vectoriel (vector space models) comme représentations distribuées de la sémantique des mots 
•	Implémenter la fréquence de co-occurrence et la PPMI
•	Comprendre différentes mesures de distance entre vecteurs de mots 
•	Explorer l’intérêt de la réduction de dimensionnalité 



## Équipe et contributions 
Veuillez indiquer la contribution effective de chaque membre de l'équipe en pourcentage et en indiquant les modules ou questions sur lesquelles chaque membre a travaillé

Luu Thien-Kim: x% (détail)

Nom Étudiant 2: x% (détail)

Nom Étudiant 3: x% (détail)



## Support de google Colab



In [210]:
# !wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !wget https://staff.fnwi.uva.nl/e.bruni/resources/MEN.tar.gz

In [211]:
# ! tar -xzf aclImdb_v1.tar.gz
# ! tar -xzf MEN.tar.gz
# ! mkdir -p vsm

In [212]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mouradyounes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mouradyounes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Librairies externes

In [213]:
from collections import Counter, defaultdict
from itertools import chain
import csv
import itertools
import numpy as np
import os
import pandas as pd

from nltk.corpus import stopwords as all_stopwords
from nltk import word_tokenize

from scipy.stats import spearmanr
from scipy.spatial.distance import euclidean, cosine
from IPython.display import display
from sklearn.decomposition import TruncatedSVD

## Valeurs globales

In [214]:
DIRNAME_ACL =  os.path.join(os.getcwd(), "aclImdb")
DIRNAME_MEN =  os.path.join(os.getcwd(), "MEN")
DIRNAME_VSM =  os.path.join(os.getcwd(), "vsm")

## 1. Prétraitement (20 points)

**a)**	Le jeu de données est séparé en deux répertoires `train/`et `test`, chacun contenant eux-mêmes deux sous-répertoires `pos/` et `neg/` pour les revues positives et négatives. Un fichier `readme` décrit plus précisément les données. Commencez par lire ces données, en gardant séparées les données d'entraînement et de test. La fonction doit mettre les mots en minuscules,  supprimer les stopwords (vous devez utiliser ceux de NLTK) et afficher le nombre total de phrases d’entrainement,  le nombre total de phrases d’entrainement positives et négatives et le nombre total de phrases de test avec le nombre total de phrases de test positives et négatives ;

In [215]:
import os
all_stopwords.words("english")
stopwords_english = set(all_stopwords.words("english"))

TRAIN_POS_DIRECTORY = DIRNAME_ACL + '/train/pos'
TRAIN_NEG_DIRECTORY = DIRNAME_ACL + '/train/neg'
TEST_POS_DIRECTORY = DIRNAME_ACL + '/test/pos'
TEST_NEG_DIRECTORY = DIRNAME_ACL + '/test/neg'

tokenizer = nltk.RegexpTokenizer(r"\w+") #enlève toutes les ponctuations des phrases

#List[List[]] telle que [[tokens du texte 1], [tokens du texte 2], ...]
trainingSet = [] 
testingSet = []

In [216]:
def getInfo() :
    train_pos_files = os.listdir(TRAIN_POS_DIRECTORY)
    train_neg_files = os.listdir(TRAIN_NEG_DIRECTORY)
    test_pos_files = os.listdir(TEST_POS_DIRECTORY)
    test_neg_files = os.listdir(TEST_NEG_DIRECTORY)
    
    files = [train_pos_files, train_neg_files, test_pos_files, test_neg_files]
    
    training_sentences_nb = 0
    training_sentences_pos_nb = 0
    training_sentences_neg_nb = 0

    testing_sentences_nb = 0
    testing_sentences_pos_nb = 0
    testing_sentences_neg_nb = 0
    
    for listOfFiles in files :
        directory = ""
        if listOfFiles == train_pos_files :
            directory = TRAIN_POS_DIRECTORY
        elif listOfFiles == train_neg_files :
            directory = TRAIN_NEG_DIRECTORY
        elif listOfFiles == test_pos_files :
            directory = TEST_POS_DIRECTORY
        elif listOfFiles == test_neg_files :
            directory = TEST_NEG_DIRECTORY
            
        for file in listOfFiles:
            with open(directory + '/' + file, "r") as f:
                data = list(f)[0].lower()       
                tokens = [token for token in tokenizer.tokenize(data) if token not in stopwords_english] #enlève toutes les ponctuations des phrases
                data = [token for token in nltk.word_tokenize(data) if token not in stopwords_english]
                data = ' '.join(data)
                
                if "train" in directory :
                    trainingSet.append(tokens)
                    if "pos" in directory : 
                        training_sentences_pos_nb += len(nltk.sent_tokenize(data))
                    elif "neg" in directory :
                        training_sentences_neg_nb += len(nltk.sent_tokenize(data))
                    
                elif "test" in directory :
                    testingSet.append(tokens)
                    if "pos" in directory : 
                        testing_sentences_pos_nb += len(nltk.sent_tokenize(data))
                    elif "neg" in directory :
                        testing_sentences_neg_nb += len(nltk.sent_tokenize(data))
            
    training_sentences_nb = training_sentences_pos_nb + training_sentences_neg_nb
    testing_sentences_nb = testing_sentences_pos_nb + testing_sentences_neg_nb
    
    print("nombre total de phrases d’entrainement : ", training_sentences_nb)
    print("nombre total de phrases d’entrainement positives : ", training_sentences_pos_nb)
    print("nombre total de phrases d’entrainement négatives : ", training_sentences_neg_nb)        
    
    print("nombre total de phrases de test : ", testing_sentences_nb)
    print("nombre total de phrases de test positives : ", testing_sentences_pos_nb)
    print("nombre total de phrases de test négatives : ", testing_sentences_neg_nb)     
    
#     return training_sentences_nb, training_sentences_pos_nb, training_sentences_neg_nb, testing_sentences_nb, testing_sentences_pos_nb, testing_sentences_neg_nb

In [217]:
getInfo()

nombre total de phrases d’entrainement :  316801
nombre total de phrases d’entrainement positives :  153934
nombre total de phrases d’entrainement négatives :  162867
nombre total de phrases de test :  310594
nombre total de phrases de test positives :  150268
nombre total de phrases de test négatives :  160326


**a)**	Créez la fonction `build_voc()` qui extrait les unigrammes de l’ensemble d’entraînement et conserve ceux qui ont une fréquence d’occurrence d'au moins 5 et imprime le nombre de mots dans le vocabulaire. Sauvegardez-le dans un fichier `vocab.txt` (un mot par ligne) dans le répertoire aclImdb.

In [219]:
def build_voc(corpus, unk_cutoff=5):
    count = 0
    dict_ = {}
    
    for tokens in trainingSet :
        for token in tokens :
            if token not in dict_ :
                dict_[token] = 0
            dict_[token] += 1
            
    newFilePath = DIRNAME_ACL + '/vocab.txt'
    with open(newFilePath, "w") as f: 
        for key in dict_ :
            if dict_[key] >= unk_cutoff :
                count += 1
                f.write(key + "\n")
                
    print(count)

In [220]:
build_voc(trainingSet)

28962

## 2. Matrices de co-occurence (30 points)

Pour les matrices de cette section, vous pourrez utiliser [des array `numpy`](https://docs.scipy.org/doc/numpy/reference/arrays.ndarray.html) ou des DataFrame [`pandas`](https://pandas.pydata.org/pandas-docs/stable/). 

Ressources utiles :  le [*quickstart tutorial*](https://numpy.org/devdocs/user/quickstart.html) de numpy et le guide [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html).

**a)** A partir des textes du corpus d’entrainement (neg/pos), vous devez construire une matrice de co-occurrence mot × mot M(w,w) qui contient les 5000 unigrammes les plus fréquents sous forme de **cadre panda**. Le contexte de co-occurrence est une fenêtre de +/-5 mots autour du mot cible. Le poids est la fréquence de co-occurrence simple. Sauvegardez votre matrice dans un fichier tp2_mat5.csv dans le répertoire vsm.

Attention, le mot lui même de doit pas être compté dans sa co-occurence. Exemple : 
Corpus: [ "I go to school every day by bus", "i go to theatre every night by bus"]

Co-occurence("every", fenetre=2) = [ (to, 2), (by, 2), (school, 1), (day, 1), (theatre, 1), (night, 1), (bus, 0), (every, 0), (go, 0). (i,0) ]

In [221]:
import re
corpus = [ "I go to school every day by bus", "i go to theatre every night by bus"]
corpus1 = [["I","go","to","school","every","day","by","bus"], ["i","go","to","theatre","every","night","by","bus"]]


In [244]:
def buildMatrix(trainingSet) :
    dict_ = {}
    words = []

    for tokens in trainingSet :
        for token in tokens :
            if token not in dict_ :
                dict_[token] = {}
            words.append(token)

        for i in range(len(tokens)) :
            frame = [i-5, i-4, i-3, i-2, i-1, i+1, i+2, i+3, i+4, i+5]
            temp = frame.copy()
            for j in range(len(frame)) :
                if frame[j] < 0 or frame[j] > len(tokens)-1 :
                    temp.remove(frame[j])
            frame = temp

            neighbouring_tokens_dict = dict_[tokens[i]]

            for index in frame :
                if tokens[i] == tokens[index]:
                    continue
                if tokens[index] not in neighbouring_tokens_dict :
                    neighbouring_tokens_dict[tokens[index]] = 0
                neighbouring_tokens_dict[tokens[index]] += 1

    c = Counter(words)
    words_to_display = c.most_common(5000)
    
    w = []
    for word in words_to_display :
        w.append(word[0])

    d = dict_.copy()

    for key in dict_ :
        if key not in w :
            d.pop(key)

    vector = [(k, v) for k, v in d.items()]

    df1 = pd.DataFrame(vector)
    df2 = pd.json_normalize(df1[1])

    df2 = df2[df1[0]].fillna(0).set_index(df1[0])
    
    return df2

import time

start = time.time()
result = buildMatrix(trainingSet)
end = time.time()
print(end - start)
result

102.74273800849915


,movie,gets,respect,sure,lot,memorable,quotes,listed,gem,imagine,...,macarthur,uwe,boll,seagal,porno,zizek,rambo,damme,prom,drivel
0,,,,,,,,,,,,,,,,,,,,,
movie,0.0,353.0,71.0,493.0,763.0,80.0,14.0,24.0,64.0,117.0,...,4.0,19.0,22.0,26.0,23.0,7.0,10.0,9.0,10.0,13.0
gets,353.0,0.0,7.0,23.0,46.0,7.0,1.0,1.0,0.0,7.0,...,1.0,0.0,1.0,1.0,2.0,1.0,3.0,3.0,2.0,1.0
respect,71.0,7.0,0.0,2.0,15.0,2.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
sure,493.0,23.0,2.0,0.0,65.0,7.0,3.0,4.0,5.0,9.0,...,0.0,0.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,1.0
lot,763.0,46.0,15.0,65.0,0.0,4.0,3.0,2.0,2.0,14.0,...,0.0,2.0,1.0,3.0,0.0,2.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zizek,7.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rambo,10.0,3.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
damme,9.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [223]:
def getVoc():
    voc = []
    voc_path = DIRNAME_ACL + '/vocab.txt'
    with open(voc_path, "r") as f: 
        data = list(f)
        for word in data:
            voc.append(word.strip('\n'))
        
    return voc
    

In [224]:
vocabulary = getVoc()

**b)** Calculez maintenant une matrice de cooccurrence mais en ajustant les fréquences basées sur la proximité du mot cible par exemple en les multipliant par 1/𝑑 où d est la distance en jetons (mots) de la cible. Sauvegardez votre matrice (toujours sous forme de cadre panda) dans un fichier tp2_mat5_scaled.csv dans le répertoire vsm.

In [243]:
def build_distance_matrix(trainingSet) :
    dict_ = {}
    words = []

    for tokens in trainingSet :
        for token in tokens :
            if token not in dict_ :
                dict_[token] = {}
            words.append(token)

        for i in range(len(tokens)) :
            frame = [i-5, i-4, i-3, i-2, i-1, i+1, i+2, i+3, i+4, i+5]
            tempFrame = frame.copy()
            for j in range(len(frame)) :
                if frame[j] < 0 or frame[j] > len(tokens)-1 :
                    tempFrame.remove(frame[j])
            frame = tempFrame

            neighbouring_tokens_dict = dict_[tokens[i]]
            for k,index in enumerate(frame):
                if tokens[i] == tokens[index]:
                    continue
                if tokens[index] not in neighbouring_tokens_dict :
                    neighbouring_tokens_dict[tokens[index]] = 0
                neighbouring_tokens_dict[tokens[index]] += 1 + 1/abs(tokens.index(tokens[i]) - tokens.index(tokens[index]))

    c = Counter(words)
    words_to_display = c.most_common(5000)
    
    w = []
    for word in words_to_display :
        w.append(word[0])

    d = dict_.copy()

    for key in dict_ :
        if key not in w :
            d.pop(key)

    vector = [(k, v) for k, v in d.items()]

    df1 = pd.DataFrame(vector)
    df2 = pd.json_normalize(df1[1])

    df2 = df2[df1[0]].fillna(0).set_index(df1[0])
    
    return df2

import time

start = time.time()
result = build_distance_matrix(trainingSet)
end = time.time()
print(end - start)
result

163.49101901054382


,movie,gets,respect,sure,lot,memorable,quotes,listed,gem,imagine,...,macarthur,uwe,boll,seagal,porno,zizek,rambo,damme,prom,drivel
0,,,,,,,,,,,,,,,,,,,,,
movie,0.000000,411.843663,82.985696,583.154166,921.418449,94.587893,17.560503,28.607931,82.299414,140.447309,...,5.553607,20.647844,24.636377,31.347149,29.751866,7.191207,10.205420,11.386255,10.954361,18.814542
gets,411.843663,0.000000,11.666667,30.202893,65.384639,10.050000,1.200000,1.500000,0.000000,9.181252,...,1.003115,0.000000,1.005376,1.007692,2.450000,1.006849,4.012500,3.073098,2.395833,1.006494
respect,82.985696,11.666667,0.000000,3.111111,21.094565,2.583333,1.250000,1.200000,0.000000,1.011765,...,0.000000,0.000000,0.000000,1.006061,0.000000,0.000000,0.000000,0.000000,0.000000,1.250000
sure,583.154166,30.202893,3.111111,0.000000,98.056881,8.903436,4.166667,6.250000,6.900000,13.202959,...,0.000000,0.000000,1.003086,3.015152,1.333333,1.019231,2.016846,0.000000,0.000000,1.250000
lot,921.418449,65.384639,21.094565,98.056881,0.000000,6.400000,5.500000,2.833333,2.583333,17.379832,...,0.000000,2.202660,1.020833,5.250000,0.000000,2.022867,1.333333,0.000000,1.009434,1.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zizek,7.191207,1.006849,0.000000,1.019231,2.022867,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
rambo,10.205420,4.012500,0.000000,2.016846,1.333333,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
damme,11.386255,3.073098,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,4.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


**c)**	Vous devez créer une fonction `pmi` qui prend le cadre panda de la matrice $M(w,w)$ et un paramètre boolean flag qui est à True lorsque l'on désire calculer PPMI et à False quand on veut calculer PMI. La fonction transforme la matrice en entrée en une matrice $M’(w,w)$ avec les valeurs PMI ou PPMI selon la valeur du paramètre booléen. La fonction retourne le nouveau cadre panda correspondant. 

Pour une matrice  $X_{m \times n}$:


$$\textbf{colsum}(X, j) = \sum_{i=1}^{m}X_{ij}$$

$$\textbf{sum}(X) = \sum_{i=1}^{m}\sum_{j=1}^{n} X_{ij}$$

$$\textbf{expected}(X, i, j) = 
\frac{
  \textbf{rowsum}(X, i) \cdot \textbf{colsum}(X, j)
}{
  \textbf{sum}(X)
}$$


$$\textbf{pmi}(X, i, j) = \log\left(\frac{X_{ij}}{\textbf{expected}(X, i, j)}\right)$$

$$\textbf{ppmi}(X, i, j) = 
\begin{cases}
\textbf{pmi}(X, i, j) & \textrm{if } \textbf{pmi}(X, i, j) > 0 \\
0 & \textrm{otherwise}
\end{cases}$$


In [245]:
import math

def pmi(df, flag=True):
    totalSum = 0
    colsum = []
    for column in df:
        columnSum = df[column].sum()
        totalSum += columnSum
        colsum.append (columnSum)

    rowsum = []
    for row in df.sum(axis=1):
        rowsum.append(row)
    
    expected = []
    for row in rowsum:
        eachRow = []
        for col in colsum:
            eachRow.append((row*col)/totalSum)
        expected.append(eachRow)
    
    expected = np.array(expected)
    dfArray = np.array(df.to_numpy())
    dfArray = np.array(dfArray ,dtype = float)
    valueInLog = np.divide(dfArray, expected, out=np.zeros_like(expected), where=dfArray!=0)
    pmi = np.log2(valueInLog, out=np.zeros_like(valueInLog), where=valueInLog!=0)
    
    if flag:
        pmi[pmi < 0] = 0
    
    return pd.DataFrame(pmi, index = df.index, columns = df.columns)

In [248]:
start = time.time()
pmi_matrix = pmi(result)
end = time.time()
print(end - start)
pmi_matrix

9.178926229476929


,movie,gets,respect,sure,lot,memorable,quotes,listed,gem,imagine,...,macarthur,uwe,boll,seagal,porno,zizek,rambo,damme,prom,drivel
0,,,,,,,,,,,,,,,,,,,,,
movie,0.000000,0.000000,0.051947,0.349341,0.402087,0.000000,0.000000,0.448420,0.393894,0.170255,...,0.000000,0.322964,0.069087,0.176375,0.657150,0.000000,0.000000,0.000000,0.000000,0.000000
gets,0.000000,0.000000,0.509233,0.000000,0.149789,0.046632,0.000000,0.000000,0.000000,0.000000,...,0.188469,0.000000,0.000000,0.000000,0.933267,0.338662,1.443376,1.800168,0.758229,0.000000
respect,0.051947,0.509233,0.000000,0.000000,1.225142,0.931302,2.651132,2.355161,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.967638,0.000000,0.000000,0.000000,0.000000,0.000000,2.401193
sure,0.349341,0.000000,0.000000,0.000000,0.842317,0.240355,1.737792,1.856859,0.709223,0.463216,...,0.000000,0.000000,0.000000,0.469336,0.126990,0.532385,1.052136,0.000000,0.000000,0.000000
lot,0.402087,0.149789,1.225142,0.842317,0.000000,0.000000,1.160443,0.279510,0.000000,0.523297,...,0.000000,0.491088,0.000000,0.476949,0.000000,0.955035,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zizek,0.000000,0.338662,0.000000,0.532385,0.955035,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
rambo,0.000000,1.443376,0.000000,1.052136,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
damme,0.000000,1.800168,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,6.065928,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


**d)** Créer les matrice PMIs et PPMIs en vous basant sur les deux matrices que vous avez déjà créée	Sauvegardez vos matrices dans un fichier tp2_mat5<\_scaled>_{pmi|ppmi}.csv toujours dans le répertoire vsm. 

(votre nom de fichier doit contenir "_scaled" s'il est formé à partir Mww_scaled et "pmi" si le flag est false "ppmi" sinon) 

## 3. Test de PPMI (20 points)

Pour le test des matrices de cooccurrences, nous allons comparer deux mesures de distance entre deux vecteurs, la distance euclidienne et la distance cosinus provenant du module [scipy.spatial.distance](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

**Distance Euclidienne**

La distance euclidienne entre deux vecteurs $u$ et $v$ de dimension $n$ est

$$\textbf{euclidean}(u, v) = 
\sqrt{\sum_{i=1}^{n}|u_{i} - v_{i}|^{2}}$$

En deux dimensions, cela correspond à la longueur de la ligne droite entre deux points.

**Distance Cosinus**


La distance cosinus entre deux vecteurs $u$ et $v$ de dimension $n$ s'écrit :

$$\textbf{cosine}(u, v) = 
1 - \frac{\sum_{i=1}^{n} u_{i} \cdot v_{i}}{\|u\|_{2} \cdot \|v\|_{2}}$$

Le terme de droite dans la soustraction mesure l'angle entre $u$ et $v$; on l'appelle la *similarité cosinus* entre $u$ et $v$.

\\

**a)**	Implémentez la fonction voisins(mot, pd, distance) qui prend un mot en entrée et une métrique de distance et qui retourne les n mots les plus similaires selon la mesure. Pour un mot w, elle ordonne tous les mots du vocabulaire en fonction de leur distance de w en utilisant la métrique de distance distance (par défaut: cosine)sur le vsm pd. Les mesures de distance à tester sont : la distance Euclidienne et la distance cosinus implantées ci-dessus.

In [230]:
def voisins(word, df, distfunc, n):
    distances = words = {}
    similary_words = []
    if(word not in voc): 
        return "The word is not in list"
    else:
        for i,word in enumerate(pmi_matrix.values):
            if distfunc == 'euclidean':
                distances[vocabulary[i]] = euclidean(df,word)
            elif distfunc == 'cosine':
                distances[vocabulary[i]] = cosine(df,word)

        common_words = Counter(words).most_common(n) # Si c des tuples recup que le mot 
        for w in common_words:
            similary_words.append(w[0])
        
    return similary_words 

**b)** En utilisant le cadre panda associé aux matrices Mww et Mww scaled, trouvez les 5 mots les plus similaires au mot « beautiful » et affichez-les, pour chacune des deux distances.

In [231]:
#  TODO : CHANGER LA MATRICE : MWW
#print("EN UTILISANT LA MATRICE MOT PAR MOT : ")
# print("Les 5 mots les plus similaires à beautiful en utilisant la distance euclidienne est : ") 
# print(voisins('beautiful', result.values[vocabulary.index('beautiful')] , 'euclidean', 5))
# print("Les 5 mots les plus similaires à beautiful en utilisant la distance cosinus est : ")
# print(voisins('beautiful', result.values[vocabulary.index('beautiful')] , 'cosine', 5))


#  TODO : CHANGER LA MATRICE : MWW_SCALED
#print("EN UTILISANT LA MATRICE MOT PAR MOT NORMALISÉE : ")
# print("Les 5 mots les plus similaires à beautiful en utilisant la distance euclidienne est : ") 
# print(voisins('beautiful', result.values[vocabulary.index('beautiful')] , 'euclidean', 5))
# print("Les 5 mots les plus similaires à beautiful en utilisant la distance cosinus est : ")
# print(voisins('beautiful', result.values[vocabulary.index('beautiful')] , 'cosine', 5))

**c)** En utilisant les cadres panda associés aux matrices PMIs, trouvez les 5 mots les plus similaires au mot « beautiful » et affichez-les, pour chacune des deux distances.

In [232]:
print("EN UTILISANT LA MATRICE PMI : ")
print("Les 5 mots les plus similaires à beautiful en utilisant la distance euclidienne est : ") 
print(voisins('beautiful', pmi_matrix.values[vocabulary.index('beautiful')] , 'euclidean', 5))
print("Les 5 mots les plus similaires à beautiful en utilisant la distance cosinus est : ")
print(voisins('beautiful', pmi_matrix.values[vocabulary.index('beautiful')] , 'cosine', 5))

EN UTILISANT LA MATRICE PMI : 
Les 5 mots les plus similaires à beautiful en utilisant la distance euclidienne est : 


IndexError: index 615 is out of bounds for axis 0 with size 11

**d)** En utilisant les cadres panda associés aux matrices PPMIs, trouvez les 5 mots les plus similaires au mot
« beautiful » et affichez-les, pour chacune des deux distances. 

In [ ]:
# # TODO : CHANGER LA MATRICE : PPMI
print("EN UTILISANT LA MATRICE PPMI : ")
# print("Les 5 mots les plus similaires à beautiful en utilisant la distance euclidienne est : ") 
# print(voisins('beautiful', ppmi_matrix.values[vocabulary.index('beautiful')] , 'euclidean', 5))
# print("Les 5 mots les plus similaires à beautiful en utilisant la distance cosinus est : ")
# print(voisins('beautiful', ppmi_matrix.values[vocabulary.index('beautiful')] , 'cosine', 5))

**e)** Que constatez-vous entre la différence de performance de la distance euclidienne et la distance cosinus ? Que constatez-vous entre les différents types de matrices de cooccurrence ?

## 4.	Réduction de dimensionnalité (20 points)

**a)** Ecrivez une fonction lsa qui prend en entrée un cadre panda pd (qui contient votre matrice / vsm) et un paramètre K (qui indique le nombre de dimensions finales), et qui applique LSA avec ce paramètre k sur la matrice et retourne le vsm réduit sous forme de cadre panda.

In [ ]:
def lsa(df, k=100):

**b)** Exécutez lsa sur les cadres panda associés à vos matrices Mww et Mww_scaled avec une dimension k=100

**c)** En utilisant les matrices de co-occurrence (de base et scalés) réduites avec LSA, trouvez les 5 mots les plus similaires au mot « beautiful » selon la distance cosinus et affichez-les.

d) En utilisant les matrices PMIs et PPMIs réduites avec lsa, trouvez les 5 mots les plus similaires au mot « beautiful » selon la distance cosinus et affichez-les

**e)** En utilisant sklearn.decomposition.TruncatedSVD, créez les matrices réduites à partir des mêmes matrices que celles de la question précédentes (la matrice pmi et la matrice pmi_scaled) Puis tester ces nouvelles matrices LSA pour trouver les 5 mots les plus similaires au mot « beautiful » 

Ici aussi, nous voulons aussi obtenir des matrices de dimension k=100

f) Commentez vos résultats

## 5. Évaluation (10 points)

Il est temps d’évaluer l’intérêt de nos modèles de vecteurs. Nous allons pour cela utiliser un ensemble de données de similarité de mots (relatedness) The MEN Test Collection, qui se trouve dans le répertoire test. L’ensemble de données contient une paire de mots avec un score de similarité attribué par des humains. En d’autres termes, une ligne (un exemple) de l’ensemble de données est de la forme : \<mot_1> \<mot_2> \<score>.

Pour aligner les distances obtenues avec vos métriques, ce score est converti en nombre réel négatif par la fonction read_test_dataset que vous avez dans le squelette du TP.

La métrique d’évaluation est le coefficient de corrélation de Spearman 𝜌 entre les scores humains et vos distances (voir https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient). 

Nous allons maintenant évaluer les différents vsm obtenus sur l'ensemble de données: MEN_dataset.


#### Fonctions pour lire le jeu de données MEN

In [ ]:
def read_test_dataset(
        src_filename,
        delimiter=','):
    with open(src_filename) as f:
        reader = csv.reader(f, delimiter=delimiter)
        for row in reader:
            w1 = row[0].strip().lower()
            w2 = row[1].strip().lower()
            score = row[2]
            score = -float(score)
            yield (w1, w2, score)

In [ ]:
# Retourne un itérable sur le jeu de données MEN
def men_dataset():
    src_filename = os.path.join(
        DIRNAME_MEN, 'MEN_dataset_natural_form_full')
    return read_test_dataset(
        src_filename, delimiter=' ')

In [ ]:
def evaluate(ds, df, distfunc=cosine):
    """
    ds : iterator
       retourne des tuples (word1, word2, score).

    df : pd.DataFrame
        le modèle vsm à évaluer

    distfunc : la mesure de distance entre vecteurs
  
    Retour: le coefficient de correlation de Spearman entre les scores de l'ensemble de données de test 
    et celui du modele vsm qui se présente sous la forme d'un cadre Panda pd avec les colonnes
    ['word1', 'word2', 'score', 'distance'].
    """
    data = []
    for w1, w2, score in ds():
        d = {'word1': w1, 'word2': w2,'score': score}
        if w1 not in df.index or w2 not in df.index:
            continue
        else:
            w1 = df.loc[w1]
            w2 = df.loc[w2] 
        d['distance'] = distfunc(w1, w2)
        data.append(d)

    data = pd.DataFrame(data)
    rho, pvalue = spearmanr(data['score'].values, b=data['distance'].values)
    return rho, data

**a)**	Testez chacun de vos modèles vsm (Matrice de base, matrice scalée, les PMIs et PPMIs et toutes les matrices LSA (de base, scalée, pmi, ppmi) en appelant la fonction evaluate avec les deux mesure de distance (euclidienne et cosinus) et affichez vos résultats dans une seule table.

**b)**	Commentez vos résultats d'évaluation